In [75]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TFAutoModelForCausalLM, AutoTokenizer, AdamWeightDecay, pipeline, create_optimizer
from transformers import DefaultDataCollator
import tensorflow as tf
from datasets import Dataset, DatasetDict, load_dataset
import plotly.express as px
import plotly.io as pio
import pandas as pd
import math
import os
import torch
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pio.renderers.default = 'notebook_connected'

In [2]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
model = TFAutoModelForCausalLM.from_pretrained("distilgpt2", pad_token_id=tokenizer.eos_token_id)

2023-02-06 20:23:00.119630: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-06 20:23:00.119685: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-06 20:23:00.119705: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ifte-HomePC): /proc/driver/nvidia/version does not exist
2023-02-06 20:23:00.120132: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFGPT2LMHeadMode

In [76]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [77]:
#lyrics = pd.read_csv('/home/ifte-home/Downloads/lyrics-data.csv')
df = pd.read_csv('/home/ifte-home/Downloads/imdb_sample/texts.csv')

In [78]:
#df = lyrics.astype('str')
df = df.astype('str')

In [79]:
#Drop the songs with lyrics too long (after more than 1024 tokens, does not work)
# df = df[df['Lyric'].apply(lambda x: len(x.split(' ')) < 350)]
df = df[df['text'].apply(lambda x: len(x.split(' ')) < 350)]

In [80]:
df['text'].values[1]

'Every once in a long while a movie will come along that will be so awful that I feel compelled to warn people. If I labor all my days and I can save but one soul from watching this movie, how great will be my joy.<br /><br />Where to begin my discussion of pain. For starters, there was a musical montage every five minutes. There was no character development. Every character was a stereotype. We had swearing guy, fat guy who eats donuts, goofy foreign guy, etc. The script felt as if it were being written as the movie was being shot. The production value was so incredibly low that it felt like I was watching a junior high video presentation. Have the directors, producers, etc. ever even seen a movie before? Halestorm is getting worse and worse with every new entry. The concept for this movie sounded so funny. How could you go wrong with Gary Coleman and a handful of somewhat legitimate actors. But trust me when I say this, things went wrong, VERY WRONG.'

In [81]:
' '.join(df['text'].values[1].split(' ')[:6])

'Every once in a long while'

In [82]:
# prompt_text=df['Lyric'][0]
prompt_text=' '.join(df['text'].values[1].split(' ')[:6])
n_seqs=1
max_length=374

In [83]:
encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
# We are encoding the text using the gpt tokenizer. The return tensors are of type "pt"
# since we are using PyTorch, not tensorflow
output_sequences = model.generate(
  input_ids=encoded_prompt,
  max_length=max_length+len(encoded_prompt), # The model has to generate something, 
  # so we add the length of the original sequence to max_length
  temperature=1.0,
  top_k=0,
  top_p=0.9,
  repetition_penalty=1.2, # To ensure that we dont get repeated phrases
  do_sample=True,
  num_return_sequences=n_seqs
) # We feed the encoded input into the model.
## Getting the output ##
if len(output_sequences.shape) > 2:
    output_sequences.squeeze_() # 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [26]:
output_sequences

<tf.Tensor: shape=(1, 375), dtype=int32, numpy=
array([[   51, 12003,   267,  8358,   304,    84,   627,  3529,   299,
        21411,   410,  3755,    11,   198,    51, 11329,   410,  3755,
           11, 38251,   198, 38351,   716,   283, 12776, 25792,   198,
           32,  3876, 12776, 25792,   198,   198,    46,   384,    84,
          716,   273, 38251,   401,    78,   334,  2611,   442,  1689,
          257,   728,    64,   198, 15681,  8083,   390,   279,  3247,
          263,   198,  5005,   279,  3247,   263,   198,   198,    36,
           84,   474,  6557,   277,  1000,    72,   401, 37365,  8358,
          299, 28749, 25211,   573,   390,   844,   283,   198,    53,
          280,   573,   443,  7785,  7201,   319,  2934,   329,   198,
        46181, 10819,   300, 35652,   198,    41,  6557,   277,   528,
          390,   256, 12003,  7201,   299, 28749,   573,   583,  1082,
          198,   198,  8491,    81, 25792,    11,   198, 37280, 10866,
           11, 23781, 20005, 

In [84]:
generated_sequences = []
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    #print(generated_sequence)
    text = tokenizer.decode(generated_sequence)
    total_sequence = (
    prompt_text + text[len(tokenizer.decode(encoded_prompt[0], 
                                            clean_up_tokenization_spaces=True, )) :]
    )
    generated_sequences.append(total_sequence)

In [85]:
generated_sequences

['Every once in a long while we start to see issues that grow out of it, the way racism and misogyny are portrayed. If there\'s one thing I think people often lose when confronted with criticism about their experience does is saying something just because they don\'t really mean what you\'re doing or would like otherwise." In 2012, she began her readjustment project through working on "the reasons why non-whites should avoid being women," which was reviewed by Shurman Hemline for MLive/Mazda eWeek from 2013 until January 2016. She spent six months here before committing herself fully back home next month after having led numerous daytime appearances as host at standup paddle wetsuit weeklies—at least three were omitted altogether this year due solely [to] design difficulty…For those unfamiliar already heeningly gay folks living behind bars say Stacey must have been waiting 30 minutes if African Americans committed so many misdemeanors every day during his July five appearance recording

In [ ]:
#Using BLEU score to compare the real sentences with the generated ones
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(len(test_set)):
  reference = test_set['True_end_lyrics'][i]
  candidate = test_set['Generated_lyrics'][i]
  scores.append(sentence_bleu(reference, candidate))

statistics.mean(scores)

In [ ]:
#Rouge score
from rouge import Rouge
rouge=Rouge()

rouge.get_scores(test_set['Generated_lyrics'], test_set['True_end_lyrics'], avg=True, ignore_empty=True)


In [64]:
class SongLyrics(Dataset):
    
#     def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):
    def __init__(self, control_code, truncate=False, gpt2_type="distilgpt2", max_length=1024):

        self.tokenizer = AutoTokenizer.from_pretrained(gpt2_type)
        self.lyrics = []

        for row in df['text']:
          self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))
                
        if truncate:
            self.lyrics = self.lyrics[:20000]
        self.lyrics_count = len(self.lyrics)
        
    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]

In [86]:
dataset = SongLyrics(df['text'], truncate=True, gpt2_type="gpt2")
# dataset = SongLyrics(df['text'], truncate=True, gpt2_type="distilgpt2")

In [87]:
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [88]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=20, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):

    acc_steps = 100
    device=torch.device("cuda")
#     device=torch.device("cpu")
    model = model.cuda()
#     model = model.cpu()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model
     

In [89]:
#Train the model on the specific data we have
model = train(dataset, model, tokenizer)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [3]:
data = load_dataset("CShorten/ML-ArXiv-Papers", split='train')
data

Using custom data configuration CShorten--ML-ArXiv-Papers-d4365de626c01fa2
Found cached dataset csv (/home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


Dataset({
    features: ['Unnamed: 0', 'Unnamed: 0.1', 'title', 'abstract'],
    num_rows: 117592
})

In [4]:
data = data.train_test_split(shuffle = True, seed = 200, test_size=0.2)

train = data["train"]
val = data["test"]

Loading cached split indices for dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-5d0c4eea58019ac3.arrow and /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-615abb6e0a05153b.arrow


In [20]:
train.features

Value(dtype='string', id=None)

In [5]:
# The tokenization function
def tokenization(data):
    tokens = tokenizer(data["abstract"], padding="max_length", truncation=True, max_length=300)
    return tokens

# Apply the tokenizer in batch mode and drop all the columns except the tokenization result
train_token = train.map(tokenization, batched = True, remove_columns=["title", "abstract", "Unnamed: 0", "Unnamed: 0.1"], num_proc=10)
val_token = val.map(tokenization, batched = True, remove_columns=["title", "abstract", "Unnamed: 0", "Unnamed: 0.1"], num_proc=10)


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-a0c7394a54c399b7.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-6b48318383812c82.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-ff4a9e8e71103fa3.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-241e78e969e7d0af.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-a4fa3f570227c730.arrow


...

Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-304e75f0f3eb6bae.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-83c5339d282e4b23.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-4bd5baf7c7d79f58.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-b8af896ce41549dd.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-6fae1b889eb53276.arrow


In [19]:
train_token[0]

{'input_ids': [220,
  23413,
  11,
  867,
  40522,
  423,
  587,
  5150,
  284,
  1332,
  262,
  17895,
  198,
  24622,
  1634,
  2694,
  286,
  17019,
  7686,
  13,
  383,
  15185,
  14805,
  39185,
  11,
  198,
  48126,
  8776,
  351,
  4277,
  8718,
  12,
  17143,
  7307,
  422,
  3210,
  8861,
  11,
  389,
  3402,
  198,
  1462,
  2038,
  12034,
  13,
  3423,
  356,
  10176,
  326,
  416,
  22124,
  1780,
  2746,
  198,
  11250,
  20074,
  355,
  4096,
  355,
  20796,
  286,
  11525,
  67,
  654,
  11,
  1903,
  12225,
  11,
  3585,
  198,
  1930,
  1859,
  11525,
  12083,
  11,
  290,
  14499,
  3602,
  16354,
  17670,
  11,
  356,
  460,
  22188,
  198,
  49453,
  262,
  2854,
  286,
  39185,
  319,
  17895,
  2276,
  1634,
  13,
  775,
  989,
  198,
  49453,
  902,
  319,
  1936,
  2968,
  40522,
  25,
  6374,
  1565,
  11,
  18551,
  48,
  11,
  4217,
  30386,
  11,
  7375,
  14313,
  11,
  290,
  39448,
  198,
  19608,
  292,
  316,
  13,
  3954,
  4981,
  2987,
  9922,
  422,

In [6]:
# Create labels as a copy of input_ids
def create_labels(text):
    text["labels"] = text["input_ids"].copy()
    return text

# Add the labels column using map()
lm_train = train_token.map(create_labels, batched=True, num_proc=10)
lm_val = val_token.map(create_labels, batched=True, num_proc=10)

Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-a153e9964835a045.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-6c60992628b6fe49.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-8ac88c0e73db5e9b.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-5ecd3ee8c8bc1f4d.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-1f3cd47cfb0ae737.arrow


...

Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-e0c9bbcc3ecdb5a5.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-5eb4731faff05739.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-956f0feafe115c90.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-19ab7ad01e17c859.arrow


Loading cached processed dataset at /home/ifte-home/.cache/huggingface/datasets/CShorten___csv/CShorten--ML-ArXiv-Papers-d4365de626c01fa2/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-565b4f5b28cdcd28.arrow


In [7]:
train_set = model.prepare_tf_dataset(
    lm_train,
    shuffle=True,
    batch_size=16
)

validation_set = model.prepare_tf_dataset(
    lm_val,
    shuffle=False,
    batch_size=16
)

In [8]:
# Setting up the learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0005,
    decay_steps=500,
    decay_rate=0.95,
    staircase=False)
    
# Exponential decay learning rate
optimizer = AdamWeightDecay(learning_rate=lr_schedule, weight_decay_rate=0.01)

In [9]:
model.compile(optimizer=optimizer)
model.summary()

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 81912576  
 r)                                                              
                                                                 
Total params: 81,912,576
Trainable params: 81,912,576
Non-trainable params: 0
_________________________________________________________________


In [10]:
# This cell is optional
from transformers.keras_callbacks import PushToHubCallback

model_name = "GPT-2"
push_to_hub_model_id = f"{model_name}-finetuned-papers"

push_to_hub_callback = PushToHubCallback(
    output_dir="/home/ifte-home/work/clm_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
    hub_token="your HF token"
)

HTTPError: Invalid user token. If you didn't pass a user token, make sure you are properly logged in by executing `huggingface-cli login`, and if you did pass a user token, double-check it's correct.

In [13]:
# Fit with callbacks
model.fit(train_set, validation_data=validation_set, epochs=2, workers=9, use_multiprocessing=True,)


Epoch 1/2


TypeError: in user code:

    File "/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/transformers/modeling_tf_utils.py", line 1557, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/keras/optimizers/optimizer_experimental/optimizer.py", line 527, in minimize
        self.apply_gradients(grads_and_vars)
    File "/home/ifte-home/venvs/exp-Env/lib/py